In [1]:
import tensorflow as tf\
    , glob, os, numpy as np, PIL, matplotlib.pyplot as plt

## Bring the train and test data in the required format.

In [46]:
def norm(arr):
    "Convert and array with range 0,255 to -1,1"
    return (arr - 127)/128

def denorm(arr):
    "Convert and array with range -1,1 to 0,255"
    return (arr*128)+127

train_dl = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=360,
    width_shift_range=0.5,
    height_shift_range=0.5,
#     brightness_range=(0, 0.5),
    shear_range=0.5,
    zoom_range=0.5,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True,
#     rescale=1,
    preprocessing_function=norm
    
).flow_from_directory(
    'data/imagenette2-160/train',
    target_size=(224, 224)
)
valid_dl = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale=1,
    preprocessing_function=norm
).flow_from_directory(
    'data/imagenette2-160/val',
    target_size=(224, 224)
)

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


# Model Building

- Sequential Model layers- Use AT LEAST 3 hidden layers with appropriate input for each. Choose the best number for hidden units and give reasons.
> We would like to take advantage of transfer learning because our dataset is not a really big dataset. For that we would like to use a backbone of resnet50 which has 50 hidden layers. After the backbone we would like to flatten the output and apply a feature pyramid network to gradually bring down the dimensions of data to that of target predictions and finally apply a softmax layer.    Reference: [a-guide-to-transfer-learning-with-keras-using-resnet50](https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b)

- Add L2 regularization to all the layers.
> 
- Add one layer of dropout at the appropriate position and give reasons.
> We will add the dropout layer at the end of all layers in the feature pyramid network but not after the final prediction jas been made.
- Choose the appropriate activation function for all the layers.
> We would be using Relu for all the hidden layers and finally a softmax layer at the end. The softmax is used at the end (Called top but) to scale the sum of confidene of all classes to 100% or 1.0 in floating point number.

### Create Resnet backbone

In [39]:
x_batch, y_batch = next(train_dl)

In [40]:
x_batch.shape

(32, 160, 160, 3)

In [41]:
res = tf.keras.applications.resnet_v2.ResNet50V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [42]:
res.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 166, 166, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 80, 80, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 82, 82, 64)   0           conv1_conv[0][0]                 
_________________________________________________________________________________________

### Create a sequential model

In [55]:
FT = tf.keras.layers.Flatten
DR = tf.keras.layers.Dropout
BN = tf.keras.layers.BatchNormalization
DN = tf.keras.layers.Dense

In [67]:
model = tf.keras.Sequential([
    res,
    FT(),
    BN(),
    
    # First Block of Feature Pyramid Network
    DN(256, 'relu'),
    DR(0.5),
    BN(),
    
    # Second Block of Feature Pyramid Network
    DN(128, 'relu'),
    DR(0.5),
    BN(),
    
    # Third Block of Feature Pyramid Network
    DN(64, 'relu'),
    DR(0.5),
    BN(),
    
    # Third Block of Feature Pyramid Network
    DN(len(train_dl.class_indices), 'softmax'),
])

## Print the model summary.

In [68]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 5, 5, 2048)        23564800  
_________________________________________________________________
flatten_3 (Flatten)          (None, 51200)             0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 51200)             204800    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               13107456  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

# Model Compilation

- Compile the model with the appropriate loss function.
- Use an appropriate optimizer. Give reasons for the choice of learning rate and its value.
- Use accuracy as metric.

In [71]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.003),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Model Training
- Train the model for an appropriate number of epochs. Print the train and validation accuracy and loss for each epoch. Use the appropriate batch size.
- Plot the loss and accuracy history graphs for both train and validation set. Print the total time taken for training.

# 6. Model Evaluation